In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('columbia_full.csv')
df.columns

Index(['FechaPlanInicioTransporte', 'RegionalDistribucion', 'IDCentro',
       'CodigoCliente', 'DocumentoTransporte', 'Ordered_vol', 'ordered_boxes',
       'Plan_Vol', 'plan_boxes'],
      dtype='object')

In [2]:
regions = pd.unique(df['RegionalDistribucion'])
n_centres = len(pd.unique(df['IDCentro']))
print(f"Regions: {regions}, Number of centres: {n_centres}")

Regions: ['Sur' 'Centro' 'Norte' 'Andes'], Number of centres: 64


In [3]:
df.groupby('CodigoCliente').size().sort_values(ascending=False) # Number of orders

CodigoCliente
13937684    231
10359686    112
11430868     97
10151328     81
10316879     69
           ... 
12596356      1
12596294      1
12596293      1
12596275      1
14323128      1
Length: 217343, dtype: int64

In [4]:
df.groupby('CodigoCliente')['ordered_boxes'].mean().sort_values(ascending=False).head(10)

CodigoCliente
14048190    5100.000000
12719444    3840.000000
12291643    3756.500000
11618884    3718.380952
12549492    3240.000000
12404538    3213.772727
12291291    3202.000000
12282136    3136.800000
13778771    3072.000000
13876309    3000.000000
Name: ordered_boxes, dtype: float64

In [5]:

medellin = (6.251290,-75.5812)
bogota = (4.6100,-74.0833)

In [6]:
from math import radians, cos, sin, asin, sqrt

def haversine_own(lon1, lat1, lon2, lat2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Earth radius in km
    return c * r


In [7]:
haversine_own(6.251290, -75.5812, 4.6100, -74.0833) # Medellin to Bogota

173.25259469854674

In [8]:
from haversine import haversine, Unit
medellin = (6.251290,-75.5812)
bogota = (4.6100,-74.0833)
# This one is correct, ours isn't
haversine(medellin, bogota, unit=Unit.KILOMETERS) # Medellin to Bogota

246.57451679533824

In [9]:
import requests

url = "https://router.project-osrm.org/route/v1/driving/-74.0833,4.6100;-75.5812,6.251290?overview=false" # driving makes it sure it's the car

response = requests.get(url)
data = response.json()

distance_km = data['routes'][0]['distance'] / 1000  # in km
duration_h = data['routes'][0]['duration'] / 3600   # in minutes

print(f"Distance: {distance_km:.2f} km")
print(f"Duration: {duration_h:.1f} h")

/Users/amelie/Library/CloudStorage/OneDrive-epfl.ch/04_Operations/project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Distance: 415.69 km
Duration: 6.4 h


In [10]:
def get_distance(frm, to):
    """Takes two points in the form of (lat, lon) and returns the distance and duration between them."""
    url = f"https://router.project-osrm.org/route/v1/driving/{frm[1]},{frm[0]};{to[1]},{to[0]}?overview=false"
    response = requests.get(url)
    data = response.json()
    distance_km = data['routes'][0]['distance'] / 1000  # in km
    duration_h = data['routes'][0]['duration'] / 3600   # in h
    return distance_km, duration_h

get_distance(bogota, medellin) # Medellin to Bogota

(415.686, 6.407722222222222)

In [25]:
import utils
utils.get_distance(bogota, medellin) # Medellin to Bogota

(415.686, 6.407722222222222)

In [11]:
test = df.head(10)

In [12]:
test['lat'] = [4.6100,6.251290] * 5
test['long'] = [-74.0833, -75.5812] * 5

/var/folders/t8/crqx384x59sbfjcs2flq1d380000gn/T/ipykernel_33162/312598344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['lat'] = [4.6100,6.251290] * 5
/var/folders/t8/crqx384x59sbfjcs2flq1d380000gn/T/ipykernel_33162/312598344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['long'] = [-74.0833, -75.5812] * 5


In [13]:
test

,FechaPlanInicioTransporte,RegionalDistribucion,IDCentro,CodigoCliente,DocumentoTransporte,Ordered_vol,ordered_boxes,Plan_Vol,plan_boxes,lat,long
0,2025-03-18,Sur,AV46,11645498,8007789767,5.370000,50.00,5.370000,50.00,4.61000,-74.0833
1,2025-03-01,Centro,AV47,12307950,8007759726,1.154175,12.02,1.154175,12.02,6.25129,-75.5812
2,2025-03-06,Centro,AV26,13212774,8007767993,0.198000,2.00,0.198000,2.00,4.61000,-74.0833
3,2025-03-31,Norte,AV39,12946586,8007816302,9.993750,102.25,9.993750,102.25,6.25129,-75.5812
4,2025-03-18,Norte,AV76,10373066,8007789124,0.723120,8.25,0.723120,8.25,4.61000,-74.0833
5,2025-03-20,Norte,AV69,10306511,8007794868,0.399000,6.00,0.399000,6.00,6.25129,-75.5812
6,2025-03-01,Norte,AV76,10383629,8007759581,1.596000,16.20,1.596000,16.20,4.61000,-74.0833
7,2025-03-20,Sur,AV18,13682698,8007794068,0.570000,6.00,0.570000,6.00,6.25129,-75.5812
8,2025-03-12,Centro,AV04,11413970,8007778595,0.891000,9.00,0.891000,9.00,4.61000,-74.0833
9,2025-03-26,Andes,AV22,13167431,8007804633,1.312000,12.00,1.312000,12.00,6.25129,-75.5812


In [15]:
from itertools import product
coords = list(set(zip(test['lat'], test['long'])))
coord_labels = [f"{lat:.5f},{lon:.5f}" for lat, lon in coords]
coord_map = dict(zip(coord_labels, coords))  # map label to (lat, lon)

# Initialize empty matrices
distance_matrix = pd.DataFrame(index=coord_labels, columns=coord_labels)
time_matrix = pd.DataFrame(index=coord_labels, columns=coord_labels)

# Compute distances and times
for from_label, to_label in product(coord_labels, repeat=2):
    from_coord = coord_map[from_label]
    to_coord = coord_map[to_label]
    dist, duration = get_distance(from_coord, to_coord)
    distance_matrix.loc[from_label, to_label] = round(dist, 2)
    time_matrix.loc[from_label, to_label] = round(duration, 2)
    

In [16]:
distance_matrix


,"6.25129,-75.58120","4.61000,-74.08330"
"6.25129,-75.58120",0.0,405.73
"4.61000,-74.08330",415.69,0.0
